# HW4 DATASCI W261: Machine Learning at Scale 

* **Name:**  Megan Jasek
* **Email:**  meganjasek@ischool.berkeley.edu
* **Class Name:**  W261-2
* **Week Number:**  4
* **Date:**  6/7/16

In [1]:
infile = "anonymous-msweb.data"
outfile = "anonymous-msweb_converted.data"
#outfile = "test_converted.data"
with open(infile, 'r') as rf, open(outfile, 'w') as wf:
    for line in rf.readlines():
        # Split the lines on commas
        items = line.split(',')
        # If the line is a customer line, then save the customer ID for later use
        # Write the line to the output file
        if items[0] == 'C':
            cust_str = items[2]
            wf.write(line)
        # If the line is a visit line, then concatenate, the original line with the
        # current customer information (the current value of cust_str) and write
        # it to the output file
        elif items[0] == 'V':
            wf.write('%s,C,%s' % (line.strip(), cust_str))
        # All other lines write directly to the output file as is
        else:
            wf.write(line)